# Road Following by Classification - Live Demo

In this notebook, we will try to make the JetBot to follow the desired road by using the trained model.

## Model Architecture
First we define a DNN model. This needs to be identical to what used for training.

In [1]:
import torch
import torch.nn as nn
device = torch.device('cuda')

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(64*7*7, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),    # dropout layer
            nn.Linear(256, 3)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)  
        return x
    
model = Model().to(device)
print(model)

Model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3136, out_features=256, bias

## Load The Best Model
Next we need to upload the `best_model.pth` in the file browser.

Then load the parameters on the model from the `best_model.pth`.

In [2]:
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

## Preprocessing Function

Now we create a function for preprocessing image data taken by the camera. This is very similar to what we have done in the collision avoidance example.

In [9]:
import cv2
import numpy as np
import torchvision

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    # x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

## Camera Instance
Now we create a camera instance.

In [4]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

image = widgets.Image(format='jpeg', width=300, height=300)
camera = Camera.instance(width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Inference
Let's try to make an inference. This process takes for a while for the first time because it needs to load a model.

In [5]:
import torch.nn.functional as F

with torch.no_grad():
    model.eval()
    x = camera.value
    x = preprocess(x)
    y = model(x)
    y = F.softmax(y, dim=1)
    y_idx = torch.argmax(y, dim=1).item()
    print(y_idx)

0


## Robot Instance
Create the robot instance to drive the motors.

In [6]:
from jetbot import Robot

robot = Robot()

## Define Actions
Create functions of move forward, turn left and turn right.

In [7]:
def move_forward():
    robot.set_motors(0.2, 0.2)

def turn_left():
    robot.set_motors(0.0, 0.2)

def turn_right():
    robot.set_motors(0.2, 0.0)
    
actions_dict = {0:"Go Forward", 1:"Turn Left", 2:"Turn Right"}

## Run JetBot
Run JetBot with a loop.

In [10]:
import time
t0 = time.time()

display(image)
steps = 1000

with torch.no_grad():
    model.eval()

    for i in range(steps):
        x = camera.value
        x = preprocess(x)
        y = model(x)
        y = nn.Softmax(dim=1)(y)
        y_idx = torch.argmax(y, dim=1).item()

        if y_idx == 0: move_forward()
        elif y_idx == 1: turn_left()
        elif y_idx == 2: turn_right()
        
        now = time.time()
        dt = now-t0
        t0 = now
        FPS = 1/dt
        
        print(f"\rStep:{i+1}/{steps}   Action:{actions_dict[y_idx]}   FPS:{FPS:.1f}", end="")

robot.stop()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Step:1000/1000   Action:Turn Left   FPS:31.3

If you are done, stop the robot and the camera.

In [ ]:
robot.stop()
camera.stop()